# Refinitiv Data Library for Python

## Content layer - Pricing stream - Opened with a chain
This Notebook demonstrates how to retrieve level 1 streaming data (such as trades and quotes) for the constituents of a chain.  

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [login](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) to the Refinitiv Developer Community portal you will get free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be very pleased to help you. 

## Some Imports to start with

In [1]:
import refinitiv.data as rd
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session._desktop_session.Definition object at 0x7ff9aba5fd10 {name='default'}>

## Retrieve data

### Get Chain consistent

In [3]:
dow = pricing.chain.Definition(name="0#.DJI").get_stream()
dow.open(with_updates=False)
dow.close()
print(dow.constituents)

['.DJI', 'AAPL.OQ', 'AMGN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WBA.OQ', 'WMT.N']


### Create a Pricing stream with chain constituents

In [4]:
pricing_stream = pricing.Definition(universe=dow.constituents, fields=["BID", "ASK"]).get_stream()

### Define and register callbacks to receive streaming data

In [5]:
df = DataFrame()

def create_dataframe(streaming_prices):
    global df
    snapshot = streaming_prices.get_snapshot()
    field_names = snapshot.columns[1:]
    instrument_names = snapshot['Instrument'].values
    df = DataFrame(index=instrument_names, columns=field_names)
    for price in streaming_prices:
        for field_name, field_value in price.get_fields().items():
            df.at[price.name, field_name] = field_value
    display(df)


def update_dataframe(streaming_prices, instrument_name, fields):
    global df
    clear_output(wait=True)
    for field_name, field_value in fields.items():
        df.at[instrument_name, field_name] = field_value
    display(df)
    
pricing_stream.on_update(update_dataframe)
pricing_stream.on_complete(create_dataframe)

<refinitiv.data.content.pricing.Stream object at 0x7ff9aba69190 {name='['.DJI', 'AAPL.OQ', 'AMGN.OQ', 'AXP.N', 'BA.N', 'CAT.N', 'CRM.N', 'CSCO.OQ', 'CVX.N', 'DIS.N', 'DOW.N', 'GS.N', 'HD.N', 'HON.OQ', 'IBM.N', 'INTC.OQ', 'JNJ.N', 'JPM.N', 'KO.N', 'MCD.N', 'MMM.N', 'MRK.N', 'MSFT.OQ', 'NKE.N', 'PG.N', 'TRV.N', 'UNH.N', 'V.N', 'VZ.N', 'WBA.OQ', 'WMT.N']'}>

### Open the stream

In [6]:
pricing_stream.open()

,ASK,BID
.DJI,36283.2,36272.52
AAPL.OQ,148.96,148.94
AMGN.OQ,213.99,213.88
AXP.N,182.64,182.54
BA.N,221.98,221.87
CAT.N,208.77,208.68
CRM.N,306.86,306.7
CSCO.OQ,58.19,58.18
CVX.N,115.26,115.24
DIS.N,176.28,176.21


### Close the stream

In [7]:
pricing_stream.close()

<OpenState.Closed: 3>

## Close the session

In [8]:
rd.close_session()